# Spectral properties of small graphs

In [ ]:
using LinearAlgebra
using Arpack
using LaTeXStrings
using SparseArrays

In [ ]:
using Plots
using GraphPlot
using GraphRecipes
using LightGraphs

In [ ]:
include("../../dev/dice_library.jl")

In [ ]:
function spectral_process(graph)
    Nvert = nv(graph)
    Nedge = sum(degree(graph))/2
    avdeg = 2Nedge/Nvert
    
    numb = 2^(Nvert - 1)
    maxs = zeros(numb)
    mins = zeros(numb)
    cuts = zeros(numb)
    bnds = zeros(numb)
    
    maxcut = 0
    maxconf = 0
    for i in 1:numb
        conf = DyNN.number_to_conf(i, Nvert)
        curcut = DyNN.cut(graph, conf)
        cuts[i] = curcut
        bnds[i] = DyNN.conf_decay(graph, conf)[1]
        if curcut > maxcut
            maxcut = curcut
            maxconf = i
        end
     end

    return (cuts, bnds)
end

In [ ]:
function maxk(a, k)
    b = partialsortperm(a, 1:k, rev=true)
    return collect(zip(b, a[b]))
end

In [ ]:
nit = 10000

N = 16
p = 0.7

k = 15

bestcuts7 = zeros(nit)
bounds7 = zeros(nit)
gaps7 = zeros(nit)
queues7 = zeros(nit)

for i in 1:nit
#    println("\n *************** \n")
    print("$i \r")
    G = DyNN.get_connected(N, p)
    (cs, bs) = spectral_process(G)
    
    bestcuts7[i] = maximum(cs)
        
    indcuts = findall(cs .== bestcuts7[i])   # looking for the most stable best cut
    bounds7[i], = findmin(bs[indcuts])
    
    qs = findall(bs .< bounds7[i])
    queues7[i] = length(qs)
    
    if queues7[i] > 0
        lbs = bs[qs] .- bounds7[i]
        gaps7[i] = minimum(lbs)
    else
        bnds = maxk(-bs, 2)
        gaps7[i] = bs[bnds[2][1]] - bounds7[i]
    end
end

spb = sortperm(bounds7)
p1 = plot(bestcuts7[spb], markershape = :circle, markersize = 2, labels = false)
p2 = plot(bounds7[spb], markershape = :circle, markersize = 2, labels = false)
p3 =  plot(gaps7[spb], markershape = :circle, markersize = 2, labels = false)
display(plot(p2, p1, p3, layout = (3, 1)))

# hc = histogram(bestcuts, normalize=:pdf)
# hb = histogram(bounds, normalize=:pdf)
hg = histogram(gaps7, normalize=:pdf)
display(plot(hg))

In [ ]:
spb = sortperm(bounds7)
p1 = scatter(bestcuts7[spb], markershape = :circle, markersize = 2, labels = false, ylab="Max.cut", guidefontsize = 18, tickfontsize = 16)
p2 = scatter(bounds7[spb], markershape = :circle, markersize = 2, labels = false, ylab="Instability", guidefontsize = 18, tickfontsize = 16)
p3 =  scatter(gaps7[spb], markershape = :circle, markersize = 2, labels = false, ylab="Sp.sep.", guidefontsize = 18, tickfontsize = 16)
display(plot(p2, p1, p3, layout = (3, 1), size = (655, 800)))
savefig("gaps-16-7.png")

In [ ]:
hg = histogram(gaps7, normalize=:pdf, labels = false, xlab = "Spectral separation", ylab="Probability", guidefontsize = 22, tickfontsize = 19)
display(plot(hg, size=(700,1000)))
savefig("histo-16-7.png")

Bonus: instabilities 

In [ ]:
plot(bounds7, gaps7, seriestype=:scatter)

In [ ]:
plot(bounds7 .+ gaps7, seriestype=:scatter)

In [ ]:
hg = histogram(gaps7.+bounds7, normalize=:pdf, labels = false)